<a href="https://colab.research.google.com/github/HammedAiye/project/blob/main/Molecular_docking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import subprocess

def install_dependencies():
    try:
        # Update package lists
        print("Updating package lists...")
        subprocess.run(["sudo", "apt", "update"], check=True)

        # Install system dependencies for Open Babel
        print("Installing dependencies...")
        dependencies = [
            "cmake",
            "g++",
            "make",
            "python3-dev",
            "libopenbabel-dev",
            "openbabel"
        ]
        subprocess.run(["sudo", "apt", "install", "-y"] + dependencies, check=True)

        print("Dependencies installed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")
        print("Please check the error and try again.")

if __name__ == "__main__":
    install_dependencies()


In [ ]:
# Install conda if not already available
!pip install -q condacolab
import condacolab
condacolab.install()

# Install RDKit using conda
!conda install -c conda-forge rdkit -y

# Install openbabel using conda
!conda install -c conda-forge openbabel -y

# Install other packages using pip
!pip install biopython py3Dmol matplotlib pandas

In [ ]:
# Install conda if not already available
!pip install -q condacolab
import condacolab
condacolab.install()

# Install RDKit using conda
!conda install -c conda-forge rdkit -y

In [43]:
from rdkit import Chem
from rdkit.Chem import Descriptors
from openbabel import openbabel
from Bio.PDB import PDBParser
import subprocess
import py3Dmol
import matplotlib.pyplot as plt

In [44]:
def lipinski_properties(smiles):
    mol = Chem.MolFromSmiles(smiles)
    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    h_donors = Descriptors.NumHDonors(mol)
    h_acceptors = Descriptors.NumHAcceptors(mol)
    return {"Molecular Weight": mw, "LogP": logp, "H Donors": h_donors, "H Acceptors": h_acceptors}

In [45]:
def prepare_ligand(cid):
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("smi", "pdb")
    mol = openbabel.OBMol()
    obConversion.ReadString(mol, f"{cid}")
    pdb_output = f"ligand_{cid}.pdb"
    obConversion.WriteFile(mol, pdb_output)
    return pdb_output

In [46]:
def prepare_receptor(pdb_id):
    parser = PDBParser()
    structure = parser.get_structure("Receptor", f"{pdb_id}.pdb")
    return structure

In [47]:
def run_docking(receptor_pdbqt, ligand_pdbqt, grid_center, grid_size):
    cmd = f"""
    prepare_receptor4.py -r {receptor_pdbqt}
    prepare_ligand4.py -l {ligand_pdbqt}
    autogrid4 -p autogrid.gpf -l grid.log
    autodock4 -p docking.dpf -l docking.log
    """
    subprocess.run(cmd, shell=True)

    # Extract results
    results = parse_docking_log("docking.log")
    return results

In [48]:
def parse_docking_log(log_file):
    with open(log_file, 'r') as file:
        lines = file.readlines()
    results = {}
    for line in lines:
        if "Binding Energy" in line:
            results["Binding Energy"] = float(line.split(":")[-1].strip())
        if "Inhibition Constant" in line:
            results["Inhibition Constant"] = float(line.split(":")[-1].strip())
        if "vdW + H-bond + desolv Energy" in line:
            results["vdw_Hbond_Desolv"] = float(line.split(":")[-1].strip())
    return results

In [49]:
def visualize_docking(protein_file, ligand_file):
    with open(protein_file, 'r') as protein:
        protein_data = protein.read()
    with open(ligand_file, 'r') as ligand:
        ligand_data = ligand.read()

    viewer = py3Dmol.view(width=800, height=600)
    viewer.addModel(protein_data, 'pdb')
    viewer.addModel(ligand_data, 'pdb')
    viewer.setStyle({'stick': {}})
    viewer.zoomTo()
    viewer.show()

In [ ]:
# Define receptor and ligands
receptor_id = "1HNY"
ligands = ["969516", "2353", "275182"]
grid_center = (0, 0, 0)  # Replace with actual center
grid_size = (60, 60, 60)
# After uploading, check if the receptor file exists in the /content directory
receptor_file_path = "/content/1hny.pdb"  # Assuming the uploaded file is named '1hny.pdb'
# Ensure the receptor file exists
if not os.path.exists(receptor_file_path):
    print(f"Error: Receptor file {receptor_file_path} not found.")
else:
    print(f"Receptor file found at {receptor_file_path}")

    # Prepare receptor function (you can define specific preparation steps)
    def prepare_receptor(pdb_file):
        parser = PDB.PDBParser(QUIET=True)
        structure = parser.get_structure(receptor_id, pdb_file)
        # You can add further receptor preparation logic here, like adding hydrogens or other processing
        return structure

    # Prepare receptor
    structure = prepare_receptor(receptor_file_path)
    print(f"Receptor {receptor_id} prepared successfully.")

    # Loop through ligands
    for ligand in ligands:
        print(f"Processing ligand CID: {ligand}")

        # Example function to get SMILES from CID (you can implement PubChem API integration here)
        def get_smiles_from_pubchem(cid):
            # Dummy SMILES return; replace with actual logic or database query
            return "CC(C)Oc1ccccc1"  # Example SMILES for a compound

        # Get SMILES from CID
        smiles = get_smiles_from_pubchem(ligand)
        print(f"SMILES for ligand {ligand}: {smiles}")

        # Example function for Lipinski properties (you can implement actual Lipinski property calculation here)
        def lipinski_properties(smiles):
            # Dummy Lipinski properties; replace with actual calculation logic
            return {
                "Molecular Weight": 350,
                "LogP": 3.0,
                "H-bond Donors": 1,
                "H-bond Acceptors": 2
            }

        # Get Lipinski properties
        lipinski = lipinski_properties(smiles)
        print("Lipinski Properties:", lipinski)

        # Example function to prepare ligand (this could convert SMILES to PDBQT format)
        def prepare_ligand(smiles):
            # Dummy preparation step; replace with actual ligand preparation logic (e.g., using RDKit)
            return f"{ligand}_ligand.pdbqt"

        # Prepare ligand
        ligand_pdb = prepare_ligand(smiles)
        print(f"Ligand {ligand} prepared as {ligand_pdb}")

        # Example docking function (replace with actual docking procedure using a tool like AutoDock Vina)
        def run_docking(receptor_file, ligand_file, grid_center, grid_size):
            # Dummy docking result; replace with actual docking results
            return f"Docking result for {ligand_file} with receptor {receptor_file}"

        # Run docking
        results = run_docking(receptor_file_path, ligand_pdb, grid_center, grid_size)
        print("Docking Results:", results)

        # Example visualization function (replace with actual visualization logic using a library like py3Dmol)
        def visualize_docking(receptor_file, ligand_file):
            # Dummy visualization step; replace with actual visualization code
            print(f"Visualizing docking of {ligand_file} with receptor {receptor_file}")

        # Visualize docking
        visualize_docking(f"{receptor_id}_docked.pdb", f"{ligand}_docked.pdb")